In [ ]:
def filter_by_time(df, time_str="17:38", window_min=5):
    df["publish_date"] = pd.to_datetime(df["publish_date"])
    df["publish_date"] = df["publish_date"].dt.tz_localize(None)
    target_date = df["publish_date"].dt.date.iloc[0]
    target_time = datetime.strptime(f"{target_date} {time_str}", "%Y-%m-%d %H:%M")

    delta = (df["publish_date"] - target_time).abs()
    return df[delta <= timedelta(minutes=window_min)].sort_values("publish_date")


In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def fetch_all_articles_for_date(target_date: str):
    url = "https://www.rbc.ru/search/ajax/"
    all_articles = []
    page = 0

    while True:
        params = {
            "project": "rbcnews",
            "category": "",
            "dateFrom": target_date,
            "dateTo": target_date,
            "page": page,
            "query": "",
            "material": "article"
        }

        response = requests.get(url, params=params)
        data = response.json()

        items = data.get("items", [])
        if not items:
            break

        all_articles.extend(items)
        page += 1

    return pd.DataFrame(all_articles)

def filter_by_time(df, time_str="17:38", window_min=5):
    df["publish_date"] = pd.to_datetime(df["publish_date"])
    df["publish_date"] = df["publish_date"].dt.tz_localize(None)

    target_date = df["publish_date"].dt.date.iloc[0]
    target_time = datetime.strptime(f"{target_date} {time_str}", "%Y-%m-%d %H:%M")

    delta = (df["publish_date"] - target_time).abs()
    return df[delta <= timedelta(minutes=window_min)].sort_values("publish_date")

target_date = "11.04.2025"
target_time_str = "17:38"

df_all = fetch_all_articles_for_date(target_date)

if df_all.empty:
    print("Нет статей за эту дату.")
else:
    df_filtered = filter_by_time(df_all, time_str=target_time_str, window_min=5)
    if df_filtered.empty:
        print("Нет статей около указанного времени.")
    else:
        print(df_filtered[["title", "publish_date", "fronturl"]])


                                                title        publish_date  \
57  ЦБ выявил мошенническую схему с использованием... 2025-04-11 17:37:26   
56             Что известно о ядерной программе Ирана 2025-04-11 17:38:11   
55  Началась встреча Путина со спецпосланником Тра... 2025-04-11 17:42:39   

                                             fronturl  
57  https://www.rbc.ru/rbcfreenews/67f91f8f9a79470...  
56  https://www.rbc.ru/base/11/04/2025/67f79a959a7...  
55  https://www.rbc.ru/rbcfreenews/67f922979a7947e...  


In [ ]:
import json

for _, row in df_filtered.iterrows():
    record = row.to_dict()

    for key, value in record.items():
        if isinstance(value, pd.Timestamp):
            record[key] = value.isoformat()

    print(json.dumps(record, ensure_ascii=False, indent=2))

{
  "id": "67f91f8f9a794704f0ebd0e2",
  "project": "РБК",
  "project_nick": "rbcnews",
  "type": "short_news",
  "category": "Финансы",
  "title": "ЦБ выявил мошенническую схему с использованием цифровых рублей",
  "body": "",
  "publish_date": "2025-04-11T17:37:26",
  "publish_date_t": 1744382246,
  "fronturl": "https://www.rbc.ru/rbcfreenews/67f91f8f9a794704f0ebd0e2",
  "picture": "https://s0.rbk.ru/v6_top_pics/media/img/1/84/347443800318841.jpeg",
  "badge": null,
  "pay_option": "free",
  "data": null,
  "_score": 0
}
{
  "id": "67f79a959a79473eebee8d50",
  "project": "РБК",
  "project_nick": "rbcnews",
  "type": "article",
  "category": "База знаний",
  "title": "Что известно о ядерной программе Ирана",
  "body": "",
  "publish_date": "2025-04-11T17:38:11",
  "publish_date_t": 1744382291,
  "fronturl": "https://www.rbc.ru/base/11/04/2025/67f79a959a79473eebee8d50",
  "picture": "https://s0.rbk.ru/v6_top_pics/media/img/1/60/347443764413601.jpeg",
  "badge": null,
  "pay_option": "fr